# The Battle of Neighbourhoods

### This notebook will be used for the devolpment of the Applied Data Science Capstone final Assignment

In [2]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Section 1: Introduction

This project will intend to approach the resolution of a common problem for many Canadians: not knowing which neighborhood would souit them better when moving or travelling from Ottawa (the country's administrative capital) to Toronto (Ontario´s state capital as well as the country´s financial centre and most populated city). Since these two cities could be considered to be the country´s "top two", and given their proximity, Canadians living in one or the other may have to often visit the other city either for financial or administrative purposes. The aim of the project is to group similar neighborhoods in both cities into clusters in order to allow anyone to recognize which neighborhoods are more similar to their own in the other city. 

To do so, I will use two main sources of Data: first the Wikipedia pages on postal codes from Toronro and from Ottawa, from where I will use BeautifulSoup to obtain Postal Codes From the different boroughs, which will then be very usefull to use the geocoder API in order to obtain their geospatial coordinates. This information will be then usefull to obtain the neighborhoods' venues by using the Foursquare API. Finally, once I obtain the venues I will be able to cluster the naighborhoods from both cities into clusters, in order to recognize which of them are similar to one another. 

## Section 2: Data: 

As stated before, I will obtain all the data that I will need for the analysis in 3 steps. First, I will obtain the Boroughs and Neighborhoods' Postal Codes by scraping the Wikipedia page for both cities. After this, I will use the geocoder API to obtain their coordinates. Finally, Foursquare will allow me to obtain each neighborhoods nearby venues, which I will analyze in the next section. 

### Part 1: Scraping and obtaining the naighborhoods dataframes

First I will start with Ottawa

In [3]:
import requests 
from bs4 import BeautifulSoup 

In [19]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_K") 
soup = BeautifulSoup(req.content,'lxml') 
table = soup.find_all('table')[0]  
df_ottawa = pd.read_html(str(table)) 

In [20]:
ot_neighborhood=pd.DataFrame(df_ottawa[0]) 
ot_neighborhood.head()

,0,1,2,3,4,5,6
0,K1AGovernment of CanadaOttawa and Gatineau off...,K2AOttawa(Highland Park / McKellar Park /Westb...,K4AOttawa(Fallingbrook),K6AHawkesbury,K7ASmiths Falls,K8APembrokeCentral and northern subdivisions,K9ACobourg
1,K1BOttawa(Blackburn Hamlet / Pine View / Sheff...,K2BOttawa(Britannia /Whitehaven / Bayshore / P...,K4BOttawa(Navan),K6BNot assigned,K7BNot assigned,K8BPembroke(Pleasant View / Fairview),K9BNot assigned
2,K1COttawa(Orleans),K2COttawa(Queensway / Copeland Park / Central ...,K4COttawa(Cumberland),K6CNot assigned,K7CCarleton Place,K8CNot assigned,K9CNot assigned
3,K1EOttawa(Queenswood),K2EOttawa(Eastern Nepean: Fisher Heights/ Park...,K4ENot assigned,K6ENot assigned,K7ENot assigned,K8ENot assigned,K9ENot assigned
4,K1GOttawa(Riverview / Hawthorne / Canterbury /...,"K2GOttawa(Centrepointe, Meadowlands, City View...",K4GNot assigned,K6GNot assigned,K7GGananoque,K8GNot assigned,K9GNot assigned


In [27]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    elif len((row.span.text).split('(')) == 1:
        cell['PostalCode'] = row.b.text
        cell['Borough'] = row.span.text
        cell['Neighborhood'] = (row.span.text)
        
        table_contents.append(cell)
    else:
        cell['PostalCode'] = row.b.text
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        
        table_contents.append(cell)

ot_df=pd.DataFrame(table_contents)

ot_df['Neighborhood']=ot_df['Neighborhood'].replace({'partly in QC':'Government of Canada, Ottawa and Gatineau offices'})

ot_df

,PostalCode,Borough,Neighborhood
0,K1A,Government of CanadaOttawa and Gatineau offices,"Government of Canada, Ottawa and Gatineau offices"
1,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P..."
2,K4A,Ottawa,Fallingbrook
3,K6A,Hawkesbury,Hawkesbury
4,K7A,Smiths Falls,Smiths Falls
...,...,...,...
69,K1W,Ottawa,"Chapel Hill South, Blackburn"
70,K2W,Ottawa,North March
71,K1X,Ottawa,South Gloucester
72,K1Y,Ottawa,"Civic Hospital, Island Park, Hintonburg, Mecha..."


In [28]:
ot_df.shape

(74, 3)

Now I will do Toronto. This section (and forecoming sections referring to Toronto) is quite similar to that done previously in this same course

In [29]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 
soup = BeautifulSoup(req.content,'lxml') 
table = soup.find_all('table')[0]  
tor_df = pd.read_html(str(table)) 

In [30]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

tor_df=pd.DataFrame(table_contents)
tor_df['Borough']=tor_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
tor_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [31]:
tor_df.shape

(103, 3)

### Part 2: Obtaining Neighborhood's coordinates

In [12]:
!pip install geocoder
import geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 10.9 MB/s eta 0:00:01


In [32]:
table_coords=[]
postal_code = ot_df["PostalCode"]
lat_lng_coords = None

for PC in postal_code:
    cell = {}
    lat_lng_coords = None    
    while(lat_lng_coords is None):
            g = geocoder.arcgis('{}, Ottawa, Ontario'.format(PC))
            lat_lng_coords = g.latlng

    cell ["PostalCode"] = PC
    cell ["latitude"] = lat_lng_coords [0]
    cell ["longitude"] = lat_lng_coords[1]


    table_coords.append(cell)
    
ot_coordinates=pd.DataFrame(table_coords)
ot_coordinates

,PostalCode,latitude,longitude
0,K1A,45.42271,-75.70321
1,K2A,45.38293,-75.76238
2,K4A,45.47932,-75.46927
3,K6A,45.42178,-75.69116
4,K7A,45.04152,-76.07971
...,...,...,...
69,K1W,45.43521,-75.51100
70,K2W,45.38051,-76.03415
71,K1X,45.28421,-75.61540
72,K1Y,45.40186,-75.73852


In [33]:
ot_coordinates.set_index("PostalCode", inplace = True)

In [34]:
ot_df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [35]:
ot_coordinates.dtypes

latitude     float64
longitude    float64
dtype: object

In [36]:
ottawa_data = ot_df.join(ot_coordinates, on='PostalCode')
ottawa_data = pd.DataFrame (ottawa_data)
ottawa_data.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
0,K1A,Government of CanadaOttawa and Gatineau offices,"Government of Canada, Ottawa and Gatineau offices",45.42271,-75.70321
1,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",45.38293,-75.76238
2,K4A,Ottawa,Fallingbrook,45.47932,-75.46927
3,K6A,Hawkesbury,Hawkesbury,45.42178,-75.69116
4,K7A,Smiths Falls,Smiths Falls,45.04152,-76.07971


Replicating procedure for Toronto

In [37]:
table_coords=[]
postal_code = tor_df["PostalCode"]
lat_lng_coords = None

for PC in postal_code:
    cell = {}
    lat_lng_coords = None    
    while(lat_lng_coords is None):
            g = geocoder.arcgis('{}, Toronto, Ontario'.format(PC))
            lat_lng_coords = g.latlng

    cell ["PostalCode"] = PC
    cell ["latitude"] = lat_lng_coords [0]
    cell ["longitude"] = lat_lng_coords[1]


    table_coords.append(cell)
    
tor_coordinates=pd.DataFrame(table_coords)
tor_coordinates

,PostalCode,latitude,longitude
0,M3A,43.75245,-79.32991
1,M4A,43.73057,-79.31306
2,M5A,43.65512,-79.36264
3,M6A,43.72327,-79.45042
4,M7A,43.66253,-79.39188
...,...,...,...
98,M8X,43.65319,-79.51113
99,M4Y,43.66659,-79.38133
100,M7Y,43.64869,-79.38544
101,M8Y,43.63278,-79.48945


In [38]:
tor_coordinates.set_index("PostalCode", inplace = True)

In [39]:
tor_df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [40]:
tor_coordinates.dtypes

latitude     float64
longitude    float64
dtype: object

In [41]:
toronto_data = tor_df.join(tor_coordinates, on='PostalCode')
toronto_data = pd.DataFrame (toronto_data)
toronto_data.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


In [44]:
data = pd.concat ([ottawa_data, toronto_data])
data

,PostalCode,Borough,Neighborhood,latitude,longitude
0,K1A,Government of CanadaOttawa and Gatineau offices,"Government of Canada, Ottawa and Gatineau offices",45.42271,-75.70321
1,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",45.38293,-75.76238
2,K4A,Ottawa,Fallingbrook,45.47932,-75.46927
3,K6A,Hawkesbury,Hawkesbury,45.42178,-75.69116
4,K7A,Smiths Falls,Smiths Falls,45.04152,-76.07971
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto Business,Enclave of M4L,43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


### Part 3: Obtaining venues using the Foursquare API

In [45]:
import json

Loading up Foursquare credentials

In [46]:
CLIENT_ID = 'ME4QF220DGKQEMHYQ2EXPSPUNOFRBIJ2ZONHJ1CODRV5LYK1' 
CLIENT_SECRET = 'FO02TIBGYIROAYOVC5X0XMD0VVVOU5FQVNXFP4XEL0DAHDIY'
VERSION = '20210525'
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ME4QF220DGKQEMHYQ2EXPSPUNOFRBIJ2ZONHJ1CODRV5LYK1
CLIENT_SECRET:FO02TIBGYIROAYOVC5X0XMD0VVVOU5FQVNXFP4XEL0DAHDIY


In [47]:
#This app will return each location's nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # creating the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
venues = getNearbyVenues(names=data['Neighborhood'],
                                   latitudes=data['latitude'],
                                   longitudes=data['longitude']
                                  )

Government of Canada, Ottawa and Gatineau offices
Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood
Fallingbrook
Hawkesbury
Smiths Falls
PembrokeCentral and northern subdivisions
Cobourg
Blackburn Hamlet, Pine View, Sheffield Glen
Britannia,Whitehaven, Bayshore, Pinecrest
Navan
Pleasant View, Fairview
Orleans
Queensway, Copeland Park, Central Park, Bel Air,Carleton Heights
Cumberland
Carleton Place
Queenswood
Eastern Nepean: Fisher Heights/ Parkwood Hills, Borden Farm, Pine Glen
Riverview, Hawthorne, Canterbury, Hunt Club Park
Centrepointe, Meadowlands, City View, Craig Henry, Tangelwood, Grenfell Glen, Davidson Heights
Gananoque
Alta Vista, Billings Bridge
Bells Corners, Arlington Woods/Redwood, Qualicum, Crystal Beach
CornwallEast
Perth
Petawawa
PeterboroughCentral
Beacon Hill, Cyrville, Carson Grove
Barrhaven
CornwallWest
PeterboroughSouth
Overbrook, Forbes, Manor Park, Viscount Alexander Park, Finter Quarries
Beaverbrook, South March
Rockland
CornwallNorth
SW Pittsburgh

In [49]:
venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Government of Canada, Ottawa and Gatineau offices",45.42271,-75.70321,Northern Lights Sound and Light Show,45.423856,-75.698520,Plaza
1,"Government of Canada, Ottawa and Gatineau offices",45.42271,-75.70321,Alt Hotel Ottawa,45.419973,-75.698948,Hotel
2,"Government of Canada, Ottawa and Gatineau offices",45.42271,-75.70321,Juice Monkey,45.419537,-75.699445,Juice Bar
3,"Government of Canada, Ottawa and Gatineau offices",45.42271,-75.70321,Stroked Ego,45.419286,-75.699698,Men's Store
4,"Government of Canada, Ottawa and Gatineau offices",45.42271,-75.70321,Caribbean Sizzler,45.419274,-75.699715,Caribbean Restaurant
...,...,...,...,...,...,...,...
5659,"Mimico NW, The Queensway West, South of Bloor,...",43.62513,-79.52681,Kingsway Boxing Club,43.627254,-79.526684,Gym
5660,"Mimico NW, The Queensway West, South of Bloor,...",43.62513,-79.52681,Tactical Products Canada,43.626801,-79.529388,Miscellaneous Shop
5661,"Mimico NW, The Queensway West, South of Bloor,...",43.62513,-79.52681,Queensway Fish & Chips,43.621720,-79.524588,Fish & Chips Shop
5662,"Mimico NW, The Queensway West, South of Bloor,...",43.62513,-79.52681,Sleep Country,43.621340,-79.526708,Mattress Store


## Section 3: Data Analysis

### Getting dummies to change from string to numerical data 

In [51]:
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

venues_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Grouping rows by neigborhood and means of occurrence for each neighborhood 

In [86]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped.head(15)

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.066667,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0
2,"Alta Vista, Billings Bridge",0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0
3,"Army, RCAF, CANCOM",0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.020000,0.0,...,0.0,0.0,0.000000,0.01,0.0,0.000000,0.0,0.0,0.0,0.0
4,Barrhaven,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0
5,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0
6,Bayview Village,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0
7,"Beacon Hill, Cyrville, Carson Grove",0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0
8,"Beaverbrook, South March",0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0
9,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0


### Obtaining each neighborhood's top 5 venues

This function will sort venues in descending order for each neighborhood

In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Chinese Restaurant,Newsagent,Skating Rink,Supermarket,Bakery
1,"Alderwood, Long Branch",Convenience Store,Gym,Pub,Performing Arts Venue,Fast Food Restaurant
2,"Alta Vista, Billings Bridge",Playground,Construction & Landscaping,Field,Ethiopian Restaurant,Event Space
3,"Army, RCAF, CANCOM",Coffee Shop,Restaurant,Hotel,Sandwich Place,Clothing Store
4,Barrhaven,Convenience Store,Pizza Place,Bus Station,Park,Women's Store
5,"Bathurst Manor, Wilson Heights, Downsview North",Men's Store,Business Service,Women's Store,Fish Market,Falafel Restaurant
6,Bayview Village,Trail,Construction & Landscaping,Park,Women's Store,Event Space
7,"Beacon Hill, Cyrville, Carson Grove",Sandwich Place,Entertainment Service,Mexican Restaurant,Café,Astrologer
8,"Beaverbrook, South March",Gun Range,Cafeteria,Field,Event Space,Falafel Restaurant
9,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Indian Restaurant,Thai Restaurant,Comfort Food Restaurant


### Classifying neighborhoods into clusters using k-means 

Note: kclusters (the number of clusters) has been changed several times in the hope of obtaining a better outcome.

In [ ]:
from sklearn.cluster import KMeans

In [89]:
# set number of clusters
kclusters = 5

cities_grouped_clustering = venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cities_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 4, 0, 4, 0, 0, 0], dtype=int32)

In [90]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cities_merged = data

cities_merged = cities_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

cities_merged.dropna (inplace = True)

cities_merged

,PostalCode,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,K1A,Government of CanadaOttawa and Gatineau offices,"Government of Canada, Ottawa and Gatineau offices",45.42271,-75.70321,0.0,Coffee Shop,Restaurant,Sandwich Place,Hotel,Juice Bar
1,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",45.38293,-75.76238,0.0,Sporting Goods Shop,Fast Food Restaurant,Electronics Store,Gym,Gym / Fitness Center
2,K4A,Ottawa,Fallingbrook,45.47932,-75.46927,0.0,Construction & Landscaping,Sandwich Place,Spa,Restaurant,Shopping Mall
3,K6A,Hawkesbury,Hawkesbury,45.42178,-75.69116,0.0,Coffee Shop,Restaurant,Hotel,Sandwich Place,Clothing Store
5,K8A,PembrokeCentral and northern subdivisions,PembrokeCentral and northern subdivisions,45.42178,-75.69116,0.0,Coffee Shop,Restaurant,Hotel,Sandwich Place,Clothing Store
...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113,0.0,Pool,Women's Store,Field,Ethiopian Restaurant,Event Space
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133,0.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar
100,M7Y,East Toronto Business,Enclave of M4L,43.64869,-79.38544,0.0,Coffee Shop,Café,Asian Restaurant,Hotel,Restaurant
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945,0.0,Bank,Italian Restaurant,Sushi Restaurant,Coffee Shop,Fast Food Restaurant


## Section 4: Data Visualization:

### Mapping the different clusters

In [64]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.13.0             |     pyhd8ed1ab_0          97 KB  conda-forge
    aiohttp-3.7.4.post0        |   py37h5e8e339_0         625 KB  conda-forge
    alsa-lib-1.2.3             |       h516909a_0         560 KB  conda-forge
   

In [68]:
from geopy.geocoders import Nominatim

In [65]:
from matplotlib import cm
from matplotlib import colors

In [73]:
address = 'Kingston, ON' #Kingston is a location lying right between Ottawa and Toronto 

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Kingston are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Kingston are 44.230687, -76.481323.


In [92]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=8)

# setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

cities_merged["Cluster Labels"].astype(int)

# adding markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cities_merged['latitude'], cities_merged['longitude'], cities_merged['Neighborhood'], cities_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


### Analysing each cluster individually

In [93]:
cities_merged.loc[cities_merged['Cluster Labels'] == 0, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Government of CanadaOttawa and Gatineau offices,0.0,Coffee Shop,Restaurant,Sandwich Place,Hotel,Juice Bar
1,Ottawa,0.0,Sporting Goods Shop,Fast Food Restaurant,Electronics Store,Gym,Gym / Fitness Center
2,Ottawa,0.0,Construction & Landscaping,Sandwich Place,Spa,Restaurant,Shopping Mall
3,Hawkesbury,0.0,Coffee Shop,Restaurant,Hotel,Sandwich Place,Clothing Store
5,PembrokeCentral and northern subdivisions,0.0,Coffee Shop,Restaurant,Hotel,Sandwich Place,Clothing Store
...,...,...,...,...,...,...,...
98,Etobicoke,0.0,Pool,Women's Store,Field,Ethiopian Restaurant,Event Space
99,Downtown Toronto,0.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar
100,East Toronto Business,0.0,Coffee Shop,Café,Asian Restaurant,Hotel,Restaurant
101,Etobicoke,0.0,Bank,Italian Restaurant,Sushi Restaurant,Coffee Shop,Fast Food Restaurant


In [94]:
cities_merged.loc[cities_merged['Cluster Labels'] == 1, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
69,West Toronto,1.0,Convenience Store,Fish & Chips Shop,Event Space,Falafel Restaurant,Farm
71,Scarborough,1.0,Convenience Store,Auto Garage,Women's Store,Fish & Chips Shop,Event Space


In [95]:
cities_merged.loc[cities_merged['Cluster Labels'] == 2, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
20,Ottawa,2.0,Playground,Construction & Landscaping,Field,Ethiopian Restaurant,Event Space
36,Ottawa,2.0,Playground,Dog Run,Escape Room,Event Space,Falafel Restaurant
40,Ottawa,2.0,Playground,Fish & Chips Shop,Event Space,Falafel Restaurant,Farm


In [96]:
cities_merged.loc[cities_merged['Cluster Labels'] == 3, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
69,Ottawa,3.0,Home Service,Women's Store,Field,Ethiopian Restaurant,Event Space
12,Scarborough,3.0,Bar,Home Service,Fish & Chips Shop,Event Space,Falafel Restaurant
50,North York,3.0,Auto Garage,Home Service,Women's Store,Fish & Chips Shop,Event Space


In [97]:
cities_merged.loc[cities_merged['Cluster Labels'] == 4, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
16,Ottawa,4.0,Yoga Studio,Restaurant,Construction & Landscaping,Park,Chinese Restaurant
27,Ottawa,4.0,Convenience Store,Pizza Place,Bus Station,Park,Women's Store
41,Ottawa,4.0,Park,Pizza Place,American Restaurant,Restaurant,Gym
70,Ottawa,4.0,Construction & Landscaping,Women's Store,Fish & Chips Shop,Event Space,Falafel Restaurant
0,North York,4.0,Construction & Landscaping,Food & Drink Shop,Park,Fast Food Restaurant,Women's Store
1,North York,4.0,Park,German Restaurant,Nail Salon,Grocery Store,Women's Store
16,York,4.0,Trail,Park,Hockey Arena,Field,Grocery Store
18,Scarborough,4.0,Gym / Fitness Center,Construction & Landscaping,Park,Women's Store,Field
27,North York,4.0,Residential Building (Apartment / Condo),Park,Entertainment Service,Ethiopian Restaurant,Event Space
35,East York/East Toronto,4.0,Intersection,Park,Home Service,Women's Store,Field
